In [1]:
import numpy as np
import matplotlib.pyplot as plt

import sympy as sp

## Things which are given / can be calculated 

In [2]:
P_d_sp = sp.symbols('P_d')
D_p_sp, D_g_sp = sp.symbols('D_p D_g')
N_p_sp, N_g_sp = sp.symbols('N_p N_g') 
C_sp, F_sp = sp.symbols('C F')
n_p_sp, n_g_sp, n_g_guess_sp = sp.symbols('n_p n_g n_g_guess')
VR_sp, VR_guess_sp, v_t_sp = sp.symbols('VR VR_guess v_t')
power_sp, lifetime_sp = sp.symbols('power lifetime')
W_t_sp = sp.symbols('W_t')


#enter given values

uk = sp.nan
all_values = {
    P_d_sp: 10,
    D_p_sp: sp.nan,
    D_g_sp: sp.nan,
    C_sp: sp.nan,
    N_p_sp: 17,
    N_g_sp: 54,
    n_p_sp: 1200,
    n_g_sp: sp.nan,
    n_g_guess_sp: 387,
    VR_sp: sp.nan,
    VR_guess_sp: sp.nan,
    v_t_sp: sp.nan,
    F_sp: sp.nan,
    power_sp: 5,
    lifetime_sp: 15000,
    W_t_sp: sp.nan,
}

# To have in case
# all_values = {
#     P_d: 10,
#     D_p: sp.nan,
#     D_g: sp.nan,
#     C: sp.nan,
#     N_p: 17,
#     N_g: 54,
#     n_p: 1200,
#     n_g: sp.nan,
#     n_g_guess: 387,
#     VR: sp.nan,
#     VR_guess: sp.nan,
#     v_t: sp.nan,
#     F: sp.nan,
#     power: sp.nan,
#     lifetime: sp.nan
# }

all_values, len(all_values)


({P_d: 10,
  D_p: nan,
  D_g: nan,
  C: nan,
  N_p: 17,
  N_g: 54,
  n_p: 1200,
  n_g: 200,
  n_g_guess: 387,
  VR: nan,
  VR_guess: nan,
  v_t: nan,
  F: nan,
  power: nan,
  lifetime: 1000,
  W_t: nan},
 16)

## Things to find from a table

some, like Y,Z can only be found later, tbd how to handle this

In [3]:
J_p = None
J_g = None

I = None

C_p = 2300 #for steel

k_o = None
k_s = None

k_B = None
k_v = None
k_R = None


#Need cycles to find, but these are not calculated until later
#im not sure the best way to handle this
Y_np = None
Y_ng = None

Z_np = None
Z_ng = None

#from formula below
k_m = None

#for C_ma
cond = "commercial Enclosed".upper()
# cond = "precision".upper()
# cond = "extra".upper()

safety_factor = 1


## Prelim calculations
The guessing on the output shaft speed is jank. Currently it is set to ceil, but idk if this has negative implications

### Create the equations

this currently gets mad bc eq 2 does not exist bc i need to round, but sympy has not let me as of yet, alas, the structure is there i think

In [4]:

eq1 = sp.Eq(VR_guess_sp, n_p_sp / n_g_guess_sp)

# eq2_do_not_use = sp.Eq(N_g_sp, sp.ceiling(N_p_sp * VR_guess_sp))

# eq2_5 = sp.Eq(temp, sp.ceiling(N_p * VR_guess, 0))

# eq2 = eq2_do_not_use.subs(temp, sp.ceiling(, 0))

eq2 = sp.Eq(N_g_sp, sp.ceiling(N_p_sp * VR_guess_sp))
eq3 = sp.Eq(VR_sp, N_g_sp / N_p_sp)

eq4 = sp.Eq(n_g_sp, n_p_sp / VR_sp)
eq5 = sp.Eq(D_p_sp, N_p_sp / P_d_sp)
eq6 = sp.Eq(D_g_sp, N_g_sp / P_d_sp)
eq7 = sp.Eq(C_sp, (N_p_sp + N_g_sp) / (2 * P_d_sp))
eq8 = sp.Eq(v_t_sp, np.pi * D_p_sp * n_p_sp / 12)
eq9 = sp.Eq(W_t_sp, 33000 * P_d_sp / v_t_sp)
eq10 = sp.Eq(F_sp, 12 / P_d_sp)




### Solve the equations 

In [5]:
#creates a thing of all the non-nan values
known_values = {}

for var, value in all_values.items():
    if value is not sp.nan:
        known_values[var] = value

#### (only solves once)

In [6]:
# # Solve equations with known values
# solutions = sp.solve([eq1, eq3, eq4, eq5, eq6, eq7, eq8], dict=True)

# # Update known values with solutions
# for solution in solutions:
#     for var, expr in solution.items():
#         if expr is not None:
#             known_values[var] = expr.subs(known_values)
            

# # Display updated solutions
# for var, value in known_values.items():
#     print(f"{var} = {value}")






#### Solve the equations multiple times bc some of them depends on others

In [7]:
# Iteratively solve equations until all variables are found

run_count = 0

while True:
    previous_known_values = known_values.copy()
    
    # Solve equations with current known values
    solutions = sp.solve([eq1, eq2, eq3, eq4, eq5, eq6, eq7, eq8, eq9, eq10], dict=True)
    
    # Update known values with solutions
    for solution in solutions:
        for var, expr in solution.items():
            if var not in known_values or known_values[var] is sp.nan:
                known_values[var] = expr.subs(known_values)
    
    # Break if no new values are found
    if known_values == previous_known_values:
        break
    else:
        run_count += 1
        print(f"Run interation: {run_count}")

    if run_count > 5:
        break

# Display updated solutions
for var, value in known_values.items():
    print(f"{var} = {value}")


Run interation: 1
P_d = 10
N_p = 17
N_g = 54
n_p = 1200
n_g = 200
n_g_guess = 387
lifetime = 1000
C = 0.00159154943091896*v_t + 16500.0*ceiling(2.99093151218033e-7*W_t*v_t**2)/(W_t*v_t)
D_g = 33000.0*ceiling(2.99093151218033e-7*W_t*v_t**2)/(W_t*v_t)
D_p = 0.00318309886183791*v_t
F = 396000.0/(W_t*v_t)
VR = 10367255.7568463*ceiling(2.99093151218033e-7*W_t*v_t**2)/(W_t*v_t**2)
VR_guess = 400/129


#### Assign each variable a value, eventually these should all be numbers

In [8]:
# assert(len(known_values) == len(all_values))
len(known_values), len(all_values)
# known_values

(13, 16)

In [9]:

for var, value in known_values.items():
    all_values[var] = value


P_d = all_values[P_d_sp]
D_p = all_values[D_p_sp]
D_g = all_values[D_g_sp]
C = all_values[C_sp]
N_p = all_values[N_p_sp]
N_g = all_values[N_g_sp]
n_p = all_values[n_p_sp]
n_g = all_values[n_g_sp]
n_g_guess = all_values[n_g_guess_sp]
VR = all_values[VR_sp]
VR_guess = all_values[VR_guess_sp]
v_t = all_values[v_t_sp]
F = all_values[F_sp]
power = all_values[power_sp]
lifetime = all_values[lifetime_sp]
W_t = all_values[W_t_sp]


### Final assorted Calculations

In [10]:
if F is sp.nan:
    print(F_sp)

# assert(F < 2 * D_p)
# assert(F > 0.5 * D_p)

## Calulating correction Factors

### Calculate $C_{pf}$

In [11]:
#C_pf
C_pf = None

if F / D_p < 0.5:
    assert(False)
    #use graph
elif F <= 1.0:
    C_pf = F / (10 * D_p) - 0.025
elif F <= 15:
    C_pf = F / (10 * D_p) - 0.0375 + 0.0125 * F

assert(C_pf is not None)

C_pf

TypeError: cannot determine truth value of Relational

### Calculate $C_{ma}$

In [ ]:
#C_ma

assert(cond is not None)

C_ma = None

if cond == "commercial enclosed".upper():
    C_ma = 0.127 + 0.0158 * F - 1.093 * 10**-4 * F**2
elif cond == "precision".upper():
    C_ma = 0.0675 + 0.0128 * F - 0.926 * 10**-4 * F**2
elif cond == "extra".upper():
    C_ma = 0.0380 + 0.0102 * F - 0.822 * 10**-4 * F**2

assert(C_ma is not None)
C_ma

In [ ]:
k_m = 1 + C_ma + C_pf

## Loading Cycles $N_{cp}, N_{cg}$

In [ ]:
#Number of cycles

N_cp = 60 * known_values[lifetime] * n_p
N_cg = 60 * known_values[lifetime] * n_g

print(f"N_cp: {N_cp:.2e}, N_cg: {N_cg:.2e}")

### *** Implement $Y_{xx}$  and $Z_{xx}$

In [ ]:

Y_np = None
Y_ng = None

Z_np = None
Z_ng = None
assert(False)

## Stresses and allowable Stresses

### Title

In [ ]:
s_tp = (W_t * P_d) / (F * J_p) * k_o * k_s * k_m * k_B * k_v
s_tp

In [ ]:
s_tG = s_tp * J_p / J_G
s_tG

### Title

In [ ]:
s_atP_Greater_than = s_tp * k_R * safety_factor / Y_np
s_atP_Greater_than

In [ ]:
s_atG_Greater_than = s_tG * k_R * safety_factor / Y_ng
s_atG_Greater_than

### Calc $S_c$

In [ ]:
S_c = C_p * np.sqrt( (W_t * k_o * k_s * k_m  * k_v) / (F * D_p * I) ) #
S_c

### FInd adjusted values of $S_c$

In [ ]:
S_acP_Greater_than = S_c * (k_R * safety_factor) / Z_np
S_acP_Greater_than

In [ ]:
S_acg_Greater_than = S_c * (k_R * safety_factor) / Z_ng
S_acg_Greater_than

## Brinell Hardness Calculations

In [ ]:
# Prompt the user for input
HB_grade = input("HB grade 1 or 2? ")

Contact_HB_p = 0
Bending_HB_p = 0

if int(HB_grade) == 1:
    Contact_HB_p = ( S_acP_Greater_than / 1000 - 29.1 ) / 0.322
    Bending_HB_p = ( s_atP_Greater_than / 1000 - 12.8 ) / 0.0773

    Contact_HB_g = ( S_acg_Greater_than / 1000 - 29.1 ) / 0.322
    Bending_HB_g = ( s_atG_Greater_than / 1000 - 12.8 ) / 0.0773

elif int(HB_grade) == 2:
    Contact_HB_p = ( S_acP_Greater_than / 1000 - 34.3 ) / 0.349
    Bending_HB_p = ( s_atP_Greater_than / 1000 - 16.4 ) / 0.102

    Contact_HB_g = ( S_acg_Greater_than / 1000 - 34.3 ) / 0.349
    Bending_HB_g = ( s_atG_Greater_than / 1000 - 16.4 ) / 0.102

else:
    assert(False)


print(f"Contact_HB_p: {Contact_HB_p:.2f} psi \nBending_HB_p: {Bending_HB_p:.2f} psi\n")

print(f"Contact_HB_g: {Contact_HB_g:.2f} psi \nBending_HB_g: {Bending_HB_g:.2f} psi")

### use tables to find the appropriate Material, and enter its brinell hardness

In [ ]:
HB = None

### Calculate actaul allowed bending and contact stresses

In [ ]:
#too lazy to implement rn

S_acp = None
S_atp = None

S_acg = None
S_atg = None
assert(False) # so i remember to implement this

### Calculate Safety factor

In [ ]:
contact_SF_p = S_acp * Y_np / (S_c * k_R)
bending_SF_p = S_atp * Y_np / (s_tp * k_R)

print(f"Contact Safety Factor (Pinion): {contact_SF_p:f} \nBending Safety Factor (Pinion): {bending_SF_p:f}\n")

contact_SF_g = S_acg * Y_np / (S_c * k_R)
bending_SF_g = S_atg * Y_np / (s_tG * k_R)

print(f"Contact Safety Factor (Gear): {contact_SF_g:.2f} \nBending Safety Factor (Gear): {bending_SF_g:.2f}")



## Power transmitting Capacity

In [ ]:
# Power transmitting capacity

P_cap = n_p * F * I / (126000 * k_o * k_s * k_m * k_v) * ( (S_acP_Greater_than * D_p * Z_np) / (safety_factor * k_R * C_p) ) ** 2

print(f"P_cap = {P_cap:.4f} HP")